<a href="https://colab.research.google.com/github/qasimzee/isic2024/blob/main/isic_2024_challenge_table_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn==1.2.2
!pip install numpy==1.26.4

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 21.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls "/content/drive/MyDrive/kaggle/isic-2024-data"

cnn.pkl			 sample_submission.csv	test-metadata.csv	  train-metadata.csv
logistic_regression.pkl  scaler.pkl		test-metadata.csv.gsheet  train-metadata-v1.csv
random_forest_model.pkl  test-image.hdf5	train-image.hdf5


In [4]:
METADATA_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/train-metadata.csv"
metadata_df = pd.read_csv(METADATA_FILE)


<ipython-input-4-6d834c4c6865>:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv(METADATA_FILE)


In [5]:
train_df, test_df = train_test_split(metadata_df, test_size=0.2, random_state=42)

Y_test = test_df['target']


In [6]:
Y = train_df["target"]

dropped_columns = ["isic_id", "target", "patient_id", "lesion_id", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "mel_mitotic_index", "mel_thick_mm", "tbp_lv_dnn_lesion_confidence"]
X = train_df.drop(columns=dropped_columns)

# Replace NaN values
X['sex'] = X['sex'].replace(pd.NA, 'unknown')
X['anatom_site_general'] = X['anatom_site_general'].replace(pd.NA, 'unknown')

X.fillna(0, inplace=True)

In [7]:
# Identify columns with mixed types

for column in X.columns:
  if X[column].apply(type).nunique() > 1:
    print(f"{column}: {X[column].apply(type).unique()}")


In [8]:
from sklearn.calibration import CalibratedClassifierCV

# Identify categorical features
categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['number']).columns

# Create a ColumnTransformer with OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_columns),  # One-hot encode categorical columns
        ('num', StandardScaler(), numerical_columns)  # Scale numerical columns
    ]
)

# Define the Logistic Regression pipeline
logreg_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(class_weight='balanced'))
])

# Define the Random Forest pipeline
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(class_weight='balanced'))
])

# Create the VotingClassifier
model = VotingClassifier(estimators=[
    ('logreg', logreg_pipeline),
    ('rf', rf_pipeline)
], voting='soft')

model.fit(X, Y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('logreg',
                              Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(transformers=[('cat',
                                                                                OneHotEncoder(drop='first'),
                                                                                Index(['sex', 'anatom_site_general', 'image_type', 'tbp_tile_type',
       'tbp_lv_location', 'tbp_lv_location_simple', 'attribution',
       'copyright_license'],
      dtype='object')),
                                                                               ('num',
                                                                                StandardScaler(),
                                                                                Index(['age_approx', 'clin_siz...
       'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_minorAxisMM',
       'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color',
       'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL',
       'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle',
       'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z'],
      dtype='object'))])),
                                              ('classifier',
                                               RandomForestClassifier(class_weight='balanced'))]))],
                 voting='soft')

In [9]:
test_df['sex'] = test_df['sex'].replace(pd.NA, 'unknown')
test_df['anatom_site_general'] = test_df['anatom_site_general'].replace(pd.NA, 'unknown')
test_df['lesion_id'] = test_df['lesion_id'].replace(pd.NA, 'unknown')
test_df['iddx_2'] = test_df['iddx_2'].replace(pd.NA, 'unknown')
test_df['iddx_3'] = test_df['iddx_3'].replace(pd.NA, 'unknown')
test_df['iddx_4'] = test_df['iddx_4'].replace(pd.NA, 'unknown')
test_df['mel_mitotic_index'] = test_df['mel_mitotic_index'].replace(pd.NA, 'unknown')

test_df.fillna(0, inplace=True)
# Identify columns with mixed types
for column in test_df.columns:
  if test_df[column].apply(type).nunique() > 1:
    print(f"{column}: {test_df[column].unique()}")


test_isic_ids = test_df['isic_id']
test_df = test_df.drop(columns=dropped_columns)
test_df.head()

,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license
278442,55.0,female,lower extremity,3.15,TBP tile: close-up,3D: XP,15.998595,8.383633,28.868481,25.256029,...,1.007245,3.890118,1.672431,0.136691,140,160.508423,354.662811,72.125671,ACEMID MIA,CC-0
215021,40.0,male,posterior torso,3.03,TBP tile: close-up,3D: XP,17.651690,12.653870,31.869840,29.712320,...,0.987079,2.747920,2.701037,0.264865,150,144.663000,1240.713000,94.129270,"Department of Dermatology, Hospital Clínic de ...",CC-BY-NC
209685,85.0,male,lower extremity,2.72,TBP tile: close-up,3D: XP,11.409180,9.934399,16.911940,19.329230,...,1.088047,1.835973,2.095205,0.239726,130,-290.056300,103.316300,63.942630,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC
29648,55.0,male,posterior torso,3.58,TBP tile: close-up,3D: XP,15.982470,12.767900,22.729350,21.831640,...,0.592206,1.047462,1.364019,0.287234,135,-81.866640,1614.268000,72.240660,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC
323386,70.0,male,posterior torso,5.52,TBP tile: close-up,3D: XP,13.933450,7.950915,16.302050,17.114270,...,2.765024,2.548177,2.200729,0.460039,130,80.211790,1107.658000,153.928100,"Department of Dermatology, Hospital Clínic de ...",CC-BY-NC


In [10]:
# Make predictions on the test data
y_pred_proba = model.predict_proba(test_df)[:, 1]  # Probability of positive class (e.g., malignant)

# Create a DataFrame to map predictions back to 'isic_id'
predictions_df = pd.DataFrame({
    'isic_id': test_isic_ids,
    'target': y_pred_proba
})


predictions_df




,isic_id,target
278442,ISIC_6973879,0.048434
215021,ISIC_5407194,0.030818
209685,ISIC_5273739,0.248595
29648,ISIC_0802250,0.067593
323386,ISIC_8084953,0.298317
...,...,...
2741,ISIC_0139843,0.057955
239797,ISIC_6027282,0.132890
243397,ISIC_6115419,0.063105
86684,ISIC_2218691,0.126561


In [11]:
from sklearn.metrics import accuracy_score

# Assuming you have already trained your logistic regression model
# model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(test_df)

# Calculate accuracy
accuracy = accuracy_score(Y_test, y_pred)

print(f'Model accuracy: {accuracy:.2f}')



Model accuracy: 1.00


In [12]:
target_one_ids = metadata_df[metadata_df['target'] == 1]['isic_id']

target_one_ids

,isic_id
387,ISIC_0082829
935,ISIC_0096034
1245,ISIC_0104229
1846,ISIC_0119495
3478,ISIC_0157834
...,...
396165,ISIC_9877311
398671,ISIC_9941636
399936,ISIC_9972649
399943,ISIC_9972877


In [13]:
predictions_for_target_one = predictions_df[predictions_df['isic_id'].isin(target_one_ids)]
ones_with_low_prob_df = predictions_for_target_one[predictions_for_target_one['target'] < 0.7]

metadata_df[metadata_df['isic_id'].isin(ones_with_low_prob_df['isic_id'])]


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
387,ISIC_0082829,1,IP_3249371,80.0,female,anterior torso,9.27,TBP tile: close-up,3D: XP,18.093368,...,IL_7881385,Malignant::Malignant epidermal proliferations:...,Malignant,Malignant epidermal proliferations,Squamous cell carcinoma in situ,NaN,NaN,NaN,NaN,72.211683
3478,ISIC_0157834,1,IP_3927284,55.0,male,posterior torso,5.29,TBP tile: close-up,3D: white,30.966662,...,IL_2429082,Malignant::Malignant adnexal epithelial prolif...,Malignant,Malignant adnexal epithelial proliferations - ...,Basal cell carcinoma,"Basal cell carcinoma, Superficial",NaN,NaN,NaN,99.960846
18518,ISIC_0527743,1,IP_9211554,65.0,female,posterior torso,5.49,TBP tile: close-up,3D: XP,21.429160,...,IL_1194885,Malignant::Malignant adnexal epithelial prolif...,Malignant,Malignant adnexal epithelial proliferations - ...,Basal cell carcinoma,"Basal cell carcinoma, Nodular",NaN,NaN,NaN,99.999880
20007,ISIC_0565021,1,IP_3026867,65.0,male,posterior torso,6.49,TBP tile: close-up,3D: white,32.303411,...,IL_2509569,Malignant::Malignant adnexal epithelial prolif...,Malignant,Malignant adnexal epithelial proliferations - ...,Basal cell carcinoma,"Basal cell carcinoma, Superficial",NaN,NaN,NaN,99.992728
25764,ISIC_0704481,1,IP_4215006,60.0,female,head/neck,2.57,TBP tile: close-up,3D: XP,26.076635,...,IL_8728864,Malignant::Malignant adnexal epithelial prolif...,Malignant,Malignant adnexal epithelial proliferations - ...,Basal cell carcinoma,"Basal cell carcinoma, Infiltrating",NaN,NaN,NaN,89.767569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393958,ISIC_9821013,1,IP_7299252,45.0,female,posterior torso,5.91,TBP tile: close-up,3D: XP,14.876350,...,IL_3295021,Malignant::Malignant melanocytic proliferation...,Malignant,Malignant melanocytic proliferations (Melanoma),Melanoma in situ,"Melanoma in situ, associated with a nevus",NaN,NaN,NaN,99.922290
395305,ISIC_9855202,1,IP_3923321,70.0,female,posterior torso,5.13,TBP tile: close-up,3D: white,24.068933,...,IL_3289028,Malignant::Malignant melanocytic proliferation...,Malignant,Malignant melanocytic proliferations (Melanoma),Melanoma in situ,"Melanoma in situ, associated with a nevus",NaN,NaN,NaN,99.969721
398671,ISIC_9941636,1,IP_7607582,60.0,female,posterior torso,5.65,TBP tile: close-up,3D: XP,21.990980,...,IL_6366837,Malignant::Malignant adnexal epithelial prolif...,Malignant,Malignant adnexal epithelial proliferations - ...,Basal cell carcinoma,"Basal cell carcinoma, Nodular",NaN,NaN,NaN,99.985530
399943,ISIC_9972877,1,IP_8877928,65.0,female,lower extremity,12.08,TBP tile: close-up,3D: white,23.465230,...,IL_7206529,Malignant::Malignant melanocytic proliferation...,Malignant,Malignant melanocytic proliferations (Melanoma),Melanoma Invasive,"Melanoma Invasive, Superficial spreading",NaN,<1/mm^2,0.6,99.786690


In [16]:
import pickle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Assume you have trained your pipeline as described before

# Save the entire pipeline to a pickle file
with open('/content/drive/MyDrive/kaggle/isic-2024-data/logistic_regression.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model pipeline saved successfully!")


Model pipeline saved successfully!
